# Описание блокнота

Данный блокнот служит для наглядного тестирования клиентского взаимодействия с API.

# 0. Импорт библиотек

In [1]:
# !pip install -r jupiter-requirements.txt

In [1]:
import pandas as pd
import requests
import os

In [2]:
# Тип вывода результатов в jupiter
from IPython.core.interactiveshell import InteractiveShell

jup_conds = ['all',                # Показывать все вызовы
             'last',               # Показывать последнюю общность вызовов (если последним будет идти цикл с вызовами, выведет все его вызовы)
             'last_expr',          # Показывать самый последний вызов
             'none',               # Ничего не показывать
             'last_expr_or_assign' # Показывать последний вызов или присваивание
            ]

InteractiveShell.ast_node_interactivity = jup_conds[0]

# 1. Проверка работоспособности API

In [3]:
def post_request(file_path):
    with open(file_path, 'rb') as data:
        request = requests.post('http://127.0.0.1:5000/read-book', files={'file': data})
        print(f'Загрузка книги. Код ответа «{request.status_code}»')
        # print(f'Данные: {request.content}')
        return request

In [4]:
base_dir = os.getcwd()

In [6]:
r = post_request(os.path.join(base_dir, 'Tests', 'Datasets', 'PE-81b_2semestr_24_01-13_04.xlsx')).json()
r

Загрузка книги. Код ответа «200»


{'file_hash': '19e390e4fd80a2b80f448e87289087af308324307b6314e84e3085c1e27383b4',
 'sheets': {'ПЕ-81б (2)': ['ПЕ-81б']}}

In [10]:
r = post_request(os.path.join(base_dir, 'Tests', 'Datasets', 'PE-92b_2_semestr.xls')).json()
r

Загрузка книги. Код ответа «200»


{'file_hash': 'ca30529fc31f721c5442fcc7a84a920235e0d8670e3b06cfa00eec932cc930fb',
 'sheets': {'ПЕ-92б': ['ПЕ-92б']}}

In [11]:
request = requests.post(f'http://127.0.0.1:5000/merge-books', json=[1, 2, 3, 4])
request.status_code

200

In [4]:
request = requests.post(f'http://127.0.0.1:5000/merge-books')
request.status_code
df = pd.DataFrame(request.json())
df.head()

200

,group,day,num,item_name,teacher,type,pdgr,date_pair,cab
0,МЕ-81б,ПН,2,День самоподготовки,доцент Неизвестных Н.Д.,рейд,общ,2021-11-01T00:00:00.000Z,НД
1,МЕ-81б,ПН,2,День самоподготовки,доцент Неизвестных Н.Д.,рейд,общ,2021-11-08T00:00:00.000Z,НД
2,МЕ-81б,ПН,2,День самоподготовки,доцент Неизвестных Н.Д.,рейд,общ,2021-11-15T00:00:00.000Z,НД
3,МЕ-81б,ПН,2,День самоподготовки,доцент Неизвестных Н.Д.,рейд,общ,2021-11-22T00:00:00.000Z,НД
4,МЕ-81б,ПН,2,День самоподготовки,доцент Неизвестных Н.Д.,рейд,общ,2021-11-29T00:00:00.000Z,НД


In [5]:
df.to_json('merged.json',   # Имя сохраняемого файла ИЛИ преобразуемый объект
          orient='records',   # Стиль JSON файла
          indent=4,           # Уровень отступов внутри файла
          force_ascii=False,  # Запись в ASCII?
          date_format='iso')  # Формат записи даты

In [20]:
request = requests.post(f'http://127.0.0.1:5000/merge-books?year=2020')
request.status_code
df = pd.DataFrame(request.json())
df.head()

200

,group,day,num,item_name,teacher,type,pdgr,date_pair,cab
0,921,ПН,1,Математика,преподаватель высшей категории Чикризова Т.М.,лекция,общ,2020-01-13T00:00:00.000Z,125 УК№5
1,921,ПН,1,Математика,преподаватель высшей категории Чикризова Т.М.,лекция,общ,2020-01-20T00:00:00.000Z,125 УК№5
2,921,ПН,1,Математика,преподаватель высшей категории Чикризова Т.М.,лекция,общ,2020-01-27T00:00:00.000Z,125 УК№5
3,921,ПН,1,Математика,преподаватель высшей категории Чикризова Т.М.,лекция,общ,2020-02-03T00:00:00.000Z,125 УК№5
4,921,ПН,1,Математика,преподаватель высшей категории Чикризова Т.М.,лекция,общ,2020-02-10T00:00:00.000Z,125 УК№5


In [21]:
df['date_pair'] = df['date_pair'].astype('datetime64')
df.head()

,group,day,num,item_name,teacher,type,pdgr,date_pair,cab
0,921,ПН,1,Математика,преподаватель высшей категории Чикризова Т.М.,лекция,общ,2020-01-13,125 УК№5
1,921,ПН,1,Математика,преподаватель высшей категории Чикризова Т.М.,лекция,общ,2020-01-20,125 УК№5
2,921,ПН,1,Математика,преподаватель высшей категории Чикризова Т.М.,лекция,общ,2020-01-27,125 УК№5
3,921,ПН,1,Математика,преподаватель высшей категории Чикризова Т.М.,лекция,общ,2020-02-03,125 УК№5
4,921,ПН,1,Математика,преподаватель высшей категории Чикризова Т.М.,лекция,общ,2020-02-10,125 УК№5


In [17]:
int(df['date_pair'].dt.year.max())

2022

In [19]:
df['date_pair'].dt.year.mean()
df['date_pair'].dt.year.min()
df['date_pair'].dt.year.max()

2021.137836353652

2021.0

2022.0

In [21]:
df['date_pair'].dt.year.value_counts()

2021.0    1570
2022.0     251
Name: date_pair, dtype: int64

In [46]:
df['group'].value_counts()

ПЕ-81б    606
ПЕ-92б    355
ИТ-81б    324
ОЕ-81б    277
МЕ-81б    263
Name: group, dtype: int64

In [23]:
file_hash = r['file_hash']
result_types = ['json', 'excel']
result_types = ['json']
for sheet_name, groups in r['sheets'].items():
    for group_name in groups:
        for result_type in result_types:
            request = requests.post(f'http://127.0.0.1:5000/parse-book/{file_hash}/{sheet_name}/{group_name}/{result_type}')
            request.content[:100]

b'<!DOCTYPE html>\n<html>\n  <head>\n    <title>sqlalchemy.exc.IntegrityError: (sqlite3.IntegrityError) U'

In [21]:
r['sheets'].keys()

dict_keys(['ПЕ-81б (2)'])

In [10]:
files = [
         os.path.join(base_dir, 'Tests', 'Datasets', 'PE-81b_2semestr_24_01-13_04.xlsx'),
         os.path.join(base_dir, 'Tests', 'Datasets', 'MITE-91_2019-2020_II_semestr.xls'),
         os.path.join(base_dir, 'test_parser.ipynb'),
         os.path.join(base_dir, 'Design', 'cr_evolve.gif'),
        ]

for file_path in files:
    r = post_request(file_path)
    if r.status_code != 200:
        print(r.content.decode('utf8'))
    else:
        r.json()
        file_hash = r.json()['file_hash']
        # f = [True, True, True, True]
        for sheet_name, groups in r.json()['sheets'].items():
            for group_name in groups:
                # request = requests.post(f'http://127.0.0.1:5000/parse-book?file_hash={file_hash}&sheet_name={sheet_name}&group_name={group_name}')
                # request = requests.post(f'http://127.0.0.1:5000/parse-book/{file_hash}/{sheet_name}/{group_name}?f1={f[0]}&f2={f[1]}&f3={f[2]}&f4={f[3]}')
                request = requests.post(f'http://127.0.0.1:5000/parse-book/{file_hash}/{sheet_name}/{group_name}')
                print(f'Парсинг. Код ответа: {request.status_code}')
                try:
                    json_data = request.json()
                    json_data[:3]
                except:
                    print('Сервак вернул не JSON')
    print()

Загрузка книги. Код ответа «200»


{'file_hash': '19e390e4fd80a2b80f448e87289087af308324307b6314e84e3085c1e27383b4',
 'sheets': {'ПЕ-81б (2)': ['ПЕ-81б']}}

Парсинг. Код ответа: 200


[{'day': 'ПН',
  'num': 1,
  'item_name': 'Сетевое программное обеспечение',
  'teacher': 'ст.преподаватель Лебедев Г.К.',
  'type': 'ЛБ',
  'pdgr': '2',
  'date_pair': '2022-01-31T00:00:00.000Z',
  'cab': '306 УК№1'},
 {'day': 'ПН',
  'num': 1,
  'item_name': 'Сетевое программное обеспечение',
  'teacher': 'ст.преподаватель Лебедев Г.К.',
  'type': 'ЛБ',
  'pdgr': '2',
  'date_pair': '2022-02-07T00:00:00.000Z',
  'cab': '306 УК№1'},
 {'day': 'ПН',
  'num': 1,
  'item_name': 'Сетевое программное обеспечение',
  'teacher': 'ст.преподаватель Лебедев Г.К.',
  'type': 'ЛБ',
  'pdgr': '2',
  'date_pair': '2022-02-14T00:00:00.000Z',
  'cab': '306 УК№1'}]


Загрузка книги. Код ответа «200»


{'file_hash': '354828d30a445f828c48657f7566703e727832eeec1cc6977147d6df851eec31',
 'sheets': {'МИТЕ-91': ['МИТЕ-91']}}

Парсинг. Код ответа: 200


[{'day': 'ПН',
  'num': 1,
  'item_name': 'Современные технологии в программировании',
  'teacher': 'доцент Обвинцев О.А.',
  'type': 'лекция',
  'pdgr': 'общ',
  'date_pair': '2020-02-10T00:00:00.000Z',
  'cab': '310 УК№1'},
 {'day': 'ПН',
  'num': 1,
  'item_name': 'Современные технологии в программировании',
  'teacher': 'доцент Обвинцев О.А.',
  'type': 'лекция',
  'pdgr': 'общ',
  'date_pair': '2020-03-23T00:00:00.000Z',
  'cab': '310 УК№1'},
 {'day': 'ПН',
  'num': 1,
  'item_name': 'Современные технологии в программировании',
  'teacher': 'доцент Обвинцев О.А.',
  'type': 'ЛБ',
  'pdgr': 'общ',
  'date_pair': '2020-03-30T00:00:00.000Z',
  'cab': '310 УК№1'}]


Загрузка книги. Код ответа «415»
Файл должен иметь расширение «.xls» или «.xlsx»

Загрузка книги. Код ответа «413»
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>413 Request Entity Too Large</title>
<h1>Request Entity Too Large</h1>
<p>The data value transmitted exceeds the capacity limit.</p>




In [11]:
pd.DataFrame(json_data).head()

,day,num,item_name,teacher,type,pdgr,date_pair,cab
0,ПН,1,Современные технологии в программировании,доцент Обвинцев О.А.,лекция,общ,2020-02-10T00:00:00.000Z,310 УК№1
1,ПН,1,Современные технологии в программировании,доцент Обвинцев О.А.,лекция,общ,2020-03-23T00:00:00.000Z,310 УК№1
2,ПН,1,Современные технологии в программировании,доцент Обвинцев О.А.,ЛБ,общ,2020-03-30T00:00:00.000Z,310 УК№1
3,ПН,2,Волоконно-оптические системы передачи,доцент Кусайкин Д.В.,лекция,общ,2020-02-10T00:00:00.000Z,101 УК№3
4,ПН,2,Волоконно-оптические системы передачи,доцент Кусайкин Д.В.,лекция,общ,2020-02-17T00:00:00.000Z,101 УК№3


In [12]:
r.headers

{'Server': 'Werkzeug/2.1.1 Python/3.9.1', 'Date': 'Mon, 25 Apr 2022 04:28:00 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '196'}

# 2. Тестирование API